In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.special
from scipy.special import zeta, polygamma, factorial
from networkx.algorithms import community
import itertools

In [52]:
def onionlikeness_parameter(G):
    
    sum=0
    k_max=max(d for (n,d) in G.degree())
        
    for k in range(k_max):
        N_k=0
        for node in G.nodes():
            if G.degree(node)==k:
                N_k+=1
        #print(N_k)
        
        S_k=0
        G_copy=G.copy()
        for node in G.nodes():
            if G.degree(node)>k:
                G_copy.remove_node(node)
        
        for x in nx.connected_component_subgraphs(G_copy):
            S=0
            for node in x.nodes():
                if x.degree(node)==k:
                    S+=1
            if S>S_k:
                S_k=S
        
        if (N_k)!=0:
            sum+=(S_k)/(N_k)
        
    onionlikeness = sum/(k_max)
    return onionlikeness

In [76]:
simulations=20
avg=0
for x in range(simulations):
    G=nx.erdos_renyi_graph(200,0.05)
    avg+=onionlikeness_parameter(G)
print(onionlikeness_parameter(G)/simulations)

0.004242857142857143


In [77]:
simulations=20
avg=0
for x in range(simulations):
    G=nx.barabasi_albert_graph(200,1)
    avg+=onionlikeness_parameter(G)
print(onionlikeness_parameter(G)/simulations)

0.003413118131868132


In [ ]:
#def optimize_partition():
 #   rho
for i in range(3):
    print(i)

In [ ]:
def core_periphery_structure():
    rho=sum(A_ij*del_ij)
    A=nx.adjacency_matrix(G)
    del_ij
    #A is adjacency matrix of graph, del matrix is optimal c/p matrix. C/p structure is such that rho is maximized.

In [ ]:
def update_graph(G, updates, original_temperature,k,a):
    onionlikeness=[]
    modularity_list=[]
    T = original_temperature
    for x in range(updates):
        list = np.random.choice(G.nodes(),2)
        i = list[0]
        j= list[1]
        k_i = G.degree(i)
        k_j = G.degree(j)
        delta = np.absolute(k_i - k_j)
        if i in range(core) and j in range(core):
            if (i,j) in G.edges():
                p=np.random.uniform()
                if p<=a*(1/(1+delta)):
                    G.remove_edge(i,j)
                    onionlikeness.append(onionlikeness_parameter(G))
                    print("OLP:",onionlikeness_parameter(G))
                    modularize(G)
                    nw_modularity=modularity(G)
                    modularity_list.append(nw_modularity)
                    print("M:",nw_modularity)
        if i>=core and j>=core:
            if (i,j) not in G.edges():
                p=np.random.uniform()
                if p<=a*delta:
                    G.add_edge(i,j)
                    onionlikeness.append(onionlikeness_parameter(G))
                    print("OLP:",onionlikeness_parameter(G))
                    modularize(G)
                    nw_modularity=modularity(G)
                    modularity_list.append(nw_modularity)
                    print("M:",nw_modularity)
        else:
            if (i,j) in G.edges():
                p=np.random.uniform()
                if p<=T:
                    G.remove_edge(i,j)
                    onionlikeness.append(onionlikeness_parameter(G))
                    print("OLP:",onionlikeness_parameter(G))
                    modularize(G)
                    nw_modularity=modularity(G)
                    modularity_list.append(nw_modularity)
                    print("M:",nw_modularity)
            
        T=T*k
    plt.plot(modularity_list)
    plt.plot(onionlikeness)

In [ ]:
#GENERATE PROBABILITY DISTRIBUTION FOR DEGREES OF SCALE-FREE NETWORK
def degree_distribution(cutoff,gamma,xmin,N):
    probabilities=np.zeros(cutoff+1)
    normalization=scipy.special.zeta(gamma)

    for i in range(cutoff):
        probabilities[i]=(i+1)**(-gamma)
        probabilities[i]/=normalization

    psum=sum(probabilities)
    remainder=1-sum(probabilities)

    probabilities[cutoff]=remainder

    degrees=np.linspace(1,cutoff+1,cutoff+1)
    degree_distribution=np.random.choice(degrees,N,p=probabilities).astype(int)

    #print(degree_distribution)
    return degree_distribution

In [ ]:
#CREATE NETWORK WITH N NODES AND NO EDGES
def onionnetwork(N):
    G=nx.Graph()
    for i in range(N):
        G.add_node(i)
    return G

In [ ]:
def node_layer_indices(G, degree_sequence):
    attr=dict()
    degrees=[]
    for x in degree_sequence:
        if x not in degrees:
            degrees.append(x)
    degrees.sort()
    
    for i in range(len(degrees)):
        mindegree=degrees[i]
        for node in G.nodes():
            if degree_sequence[node]==mindegree:
                nodeDict=dict()
                nodeDict["layer"]=i
                attr[node]=nodeDict
    nx.set_node_attributes(G,attr)
    return(G)            

In [ ]:
#JOIN "STUBS" OF NODE LINKS
def join_stubs(G,distribution,error,a):
    stubcount=np.copy(distribution)
    for i in G.nodes():
        for j in G.nodes():
            if i!=j:
                if stubcount[i]>0 and stubcount[j]>0:
                    #calculate probability of joining: 
                    diff=np.absolute(G.nodes[i]["layer"]-G.nodes[j]["layer"])
                    p=1/(a*(1+diff))
                    x=np.random.uniform()                        
                    #if random number smaller than p, join nodes, decrease stubcounts
                    if x<=p:
                        G.add_edge(i,j)
                        stubcount[i]=stubcount[i]-1
                        stubcount[j]=stubcount[j]-1
    
    while sum(stubcount)>error:
        stubs=[]
        edgelist=[]
        for i in G.nodes():
            if stubcount[i]>0:
                stubs.append(i)
        for x in G.edges():
            edgelist.append(x)
        swap=np.random.choice(stubs,2)
        max=len(edgelist)
        index=np.random.randint(0,max)
        swapedge=edgelist[index]
        node1=swap[0]
        node2=swap[1]
        node3=swapedge[0]
        node4=swapedge[1]
        G.remove_edge(node3,node4)
        #print("removed:",(node1,node2))
        G.add_edge(node3,node1)
        G.add_edge(node4,node2)
        stubcount[node1]=stubcount[node1]-1
        stubcount[node2]=stubcount[node2]-1
        #print(sum(stubcount))
                    
    return G

In [ ]:
#girvan-newman algorithm for modularization
def modularize(network):
    communities_generator = community.girvan_newman(network)
    #plt.figure(figsize=(5,5))
    #nx.draw(network, node_size=15)
    
    comp = community.girvan_newman(network)
    for communities in itertools.islice(comp, 5):
        clusters=list(sorted(c) for c in communities)

    modulenumber=len(list(clusters))
    modulelist=list()
    
    for i in range(len(list(clusters))):
        modulelist.append(clusters[i])
    
    attr=dict()
    for i in range(modulenumber):
        for node in modulelist[i]:
            nodeDict=dict()
            nodeDict["cluster"]=i
            attr[node]=nodeDict

    nx.set_node_attributes(network,attr)

In [ ]:
#newman's definition of modularity is used
def modularity(network):
   
    m=nx.number_of_edges(network) #number of edges in network
    N=nx.number_of_nodes(network) #number of nodes in network
    partialsum=0
    for i in range(N):
        for j in range(N):
            if (i,j) in network.edges():
                A=1
            else:
                A=0
                
            if network.nodes[i]["cluster"]==network.nodes[j]["cluster"]:
                partialsum+=(A)-(((network.degree(i)*network.degree(j)))/(2*m))
    
    return partialsum/(2*m)

In [ ]:
nodes=200
core=20
#create graphs
G=nx.Graph()
for x in range(nodes):
    for y in range(nodes):
        if x in range(core) or y in range(core):
            G.add_edge(x,y)
#nx.draw(G, node_size=20)
onionlikeness_list=update_graph(G,500,20,0.8,1)
#nx.draw(G_new, node_size=20)



In [ ]:
plt.plot(onionlikeness_list)

In [ ]:
plt.plot(modularity_list)

In [ ]:
#plot onionlikeness parameter and core/periphery coefficient at each step
for a in [0.1,1,10,100]:
    for e in [20,30,50,100]:
        for c in [50,100,200,500]:
            degrees=degree_distribution(c,2,2,200)
            onion=onionnetwork(200)
            node_layer_indices(onion,degrees)
            join_stubs(onion, degrees, e, a)
            print("generated onion network",a , e, c, onionlikeness_parameter(onion))